In [1]:
# Multilayer Perceptron (MLP) for multi-class softmax classification:
# modified from 
# https://keras.io/getting-started/sequential-model-guide/#multilayer-perceptron-mlp-for-multi-class-softmax-classification

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD, Adam

import random

C:\Users\ortci\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np

n_samples = 1000000
n_partitions = 10

# Generate tasks and partitions
partition_data = np.random.random((n_samples, n_partitions)) # partition data generation
n_outputs = 2 * n_partitions
#X = np.zeros((n_samples, n_partitions + 1))             # initialize input layer
fit_data = np.zeros((n_samples, n_partitions))
y = np.zeros((n_samples, n_outputs))                 # initialize outputs layer for training 

task_data = np.zeros((n_samples, 1))                         # initialize task list

for i in range (0, n_samples):
    
    partitions = partition_data[i]
    task = random.uniform(0, partitions.max())
    task_data[i] = task
    
    for j in range (0, n_partitions):
        current_fit = partitions[j] - task
        y[i,j] = abs(current_fit)
        if current_fit < 0:
            y[i,j+n_partitions] = 0
            fit_data[i][j] = 0
        else:
            y[i,j+n_partitions] = 1
            fit_data[i][j] = 1
            
X = np.hstack((task_data,partition_data, fit_data))

In [3]:
# split data between train and test set 
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [4]:
import tensorflow as tf

model = Sequential()
n_hidden_units = 1000
# Dense(n_hidden_units) is a fully-connected layer with n_hidden_units hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 10-dimensional vectors.
model.add(Dense(n_hidden_units, input_dim=2*n_partitions+1))

model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(n_hidden_units))

model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(n_outputs))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.load_weights('MLP_relu_softmax_21_inputs_20_outputs_v1_wts.h5')

# optimizer options
sgd = SGD(lr=0.03, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = tf.train.RMSPropOptimizer(0.008)
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [5]:
model.compile(loss='mean_squared_error',
              optimizer=sgd,
              metrics=['mae'])

In [6]:
batchsize = 500

model.fit(X_train, y_train,
          epochs=1,
          batch_size=batchsize)
score = model.evaluate(X_test, y_test, batch_size=batchsize)
print(score)

'''
Why is the training loss much higher than the testing loss?

A Keras model has two modes: training and testing. Regularization mechanisms, such as Dropout and 
L1/L2 weight regularization, are turned off at testing time. Besides, the training loss is the average 
of the losses over each batch of training data. Because your model is changing over time, the loss over 
the first batches of an epoch is generally higher than over the last batches. On the other hand, the 
testing loss for an epoch is computed using the model as it is at the end of the epoch, resulting in a lower loss.
'''

Epoch 1/1
200000/200000 [==============================] - 7s 36us/step
[0.0017096545596723444, 0.01680312376935035]


'\nWhy is the training loss much higher than the testing loss?\n\nA Keras model has two modes: training and testing. Regularization mechanisms, such as Dropout and \nL1/L2 weight regularization, are turned off at testing time. Besides, the training loss is the average \nof the losses over each batch of training data. Because your model is changing over time, the loss over \nthe first batches of an epoch is generally higher than over the last batches. On the other hand, the \ntesting loss for an epoch is computed using the model as it is at the end of the epoch, resulting in a lower loss.\n'

In [7]:
from keras.models import load_model

model.save('MLP_relu_softmax_21_inputs_20_outputs_v1.h5')  # creates a HDF5 file 'my_model.h5'
model.save_weights('MLP_relu_softmax_21_inputs_20_outputs_v1_wts.h5')

'''
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('MLP_Multiclass_softmax_10_inputs.h5')
model.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5') # for same architecture
model.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5', by_name=True) # for different architecture
'''

"\ndel model  # deletes the existing model\n\n# returns a compiled model\n# identical to the previous one\nmodel = load_model('MLP_Multiclass_softmax_10_inputs.h5')\nmodel.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5') # for same architecture\nmodel.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5', by_name=True) # for different architecture\n"

In [8]:
y_pred = model.predict(X_test, batch_size=batchsize)

In [9]:
# find cases for boolean fit misclassification
misclass_set = []
for i in range(0,100):
    for j in range(n_partitions,2*n_partitions):
        if (np.around(y_pred[i][j],0) != y_test[i][j]).any():
            misclass_set.append(i)
            break
num_misclass = len(misclass_set)
print(num_misclass)

0


In [10]:
# show results of specific case
test = 2

import pandas as pd

test_df = pd.DataFrame(np.vstack((X_test[test][1:n_partitions+1],
                                  np.full((1,10),X_test[test][0]),
                                  y_test[test][0:n_partitions],
                                  y_pred[test][0:n_partitions],
                                  y_test[test][n_partitions:2*n_partitions],
                                  y_pred[test][n_partitions:2*n_partitions])))
test_df.index = ("Partition_AF",
                 "Task_Size",
                 "Actual_Part_RAF", # abs(remaining AF)
                 "Pred_Part_RAF",   # abs(remaining AF)
                 "Actual_Part_Boolean_Fit", # actual fit
                 "Pred_Part_Boolean_Fit")   # actual fit
test_df

,0,1,2,3,4,5,6,7,8,9
Partition_AF,0.209297,0.528119,0.117762,0.341980,0.403981,0.185919,0.135160,0.852869,0.959801,0.824794
Task_Size,0.938815,0.938815,0.938815,0.938815,0.938815,0.938815,0.938815,0.938815,0.938815,0.938815
Actual_Part_RAF,0.729518,0.410696,0.821053,0.596835,0.534834,0.752897,0.803655,0.085946,0.020986,0.114021
Pred_Part_RAF,0.707821,0.443022,0.773776,0.616823,0.568455,0.727837,0.244432,0.067354,0.073135,0.074868
Actual_Part_Boolean_Fit,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
Pred_Part_Boolean_Fit,0.002606,0.000820,0.002957,-0.001744,0.001496,0.002228,-0.000340,0.000795,1.005083,0.000834
